In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import pandas as pd
import re
import numpy as np
from sklearn import metrics


In [3]:
path = './result'
file_list = os.listdir(path)

In [4]:
def sc_calc_acc_condition_with_temp_with_sc(llm_model, few_shot_n, test_n, q_src_yn, ver, p_ver, sc_num, temp, excel_ver):
    tmp = pd.DataFrame()
    df_eval = pd.DataFrame()
    acc_list = []
    path = './result'
    # ./result/sc_l_result_4_15_Y_30_sys_prompt8_0.01_ver1_0.csv
    file_list = os.listdir(path)
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}_{ver}_{p_ver}_{sc_num}_{temp}_{excel_ver}')]
    opt_file = [x for x in opt_file if x.endswith(f'.csv')]

    df = pd.DataFrame()

    
    if len(opt_file)>0 : 
        for f in opt_file:
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            tmp = tmp.dropna()

            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp = tmp[tmp['o_result'].isin(['1', '0', '2'])]

            
            gold_df = tmp[['id', 'gold']].drop_duplicates()
            chk_cnt = tmp.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'question']]
            chk_cnt = chk_cnt.rename(columns = {'question': 'cnt'})
            chk_cnt = chk_cnt[chk_cnt['cnt'] == sc_num]
            chk_cnt = chk_cnt.sort_values(by = ['id', 'cnt'], ascending=[True, False]).groupby(['id']).head(1)
            df_eval = pd.merge(gold_df, chk_cnt, on = ['id'])

            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            acc_list.append(acc)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        y_true = df['o_result']
        y_pred = df['gold']
        print(metrics.classification_report(y_true, y_pred, digits=3))
        
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)
        return acc_list


In [ ]:
# Process(target=task, args=('l', 1, 10, 'Y', 10, 'sys_prompt10', 3, 0.01, 'ver5'))
# Process(target=task, args=('l', 2, 10, 'Y', 10, 'sys_prompt10', 3, 0.01, 'ver5'))
# Process(target=task, args=('l', 4, 10, 'Y', 10, 'sys_prompt10', 5, 0.01, 'ver5'))

In [5]:
list_ =         sc_calc_acc_condition_with_temp_with_sc('l', 1, 10, 'Y', 10, 'sys_prompt10', 3,  0.01, 'ver5')
print(list_)

              precision    recall  f1-score   support

           0      0.885     0.767     0.821        30
           1      0.795     0.912     0.849        34
           2      1.000     0.800     0.889         5

    accuracy                          0.841        69
   macro avg      0.893     0.826     0.853        69
weighted avg      0.849     0.841     0.840        69

l_result_1_10_Y :  84.05797101449275
[np.float64(71.42857142857143), np.float64(50.0), np.float64(100.0), np.float64(88.88888888888889), np.float64(60.0), np.float64(100.0), np.float64(100.0), np.float64(88.88888888888889), np.float64(80.0), np.float64(87.5)]


In [6]:
list_ =         sc_calc_acc_condition_with_temp_with_sc('l', 2, 10, 'Y', 10, 'sys_prompt10', 3,  0.01, 'ver5')
print(list_)

              precision    recall  f1-score   support

           0      0.935     0.806     0.866        36
           1      0.769     0.938     0.845        32
           2      1.000     0.714     0.833         7

    accuracy                          0.853        75
   macro avg      0.902     0.819     0.848        75
weighted avg      0.871     0.853     0.854        75

l_result_2_10_Y :  85.33333333333334
[np.float64(88.88888888888889), np.float64(100.0), np.float64(75.0), np.float64(85.71428571428571), np.float64(62.5), np.float64(100.0), np.float64(75.0), np.float64(100.0), np.float64(100.0), np.float64(71.42857142857143)]


In [7]:
list_ =         sc_calc_acc_condition_with_temp_with_sc('l', 4, 10, 'Y', 10, 'sys_prompt10', 5,  0.01, 'ver5')
print(list_)

              precision    recall  f1-score   support

           0      0.963     0.929     0.945        28
           1      0.867     0.963     0.912        27
           2      1.000     0.667     0.800         6

    accuracy                          0.918        61
   macro avg      0.943     0.853     0.886        61
weighted avg      0.924     0.918     0.916        61

l_result_4_10_Y :  91.80327868852459
[np.float64(100.0), np.float64(100.0), np.float64(87.5), np.float64(85.71428571428571), np.float64(100.0), np.float64(100.0), np.float64(90.0), np.float64(80.0), np.float64(100.0), np.float64(66.66666666666666)]
